# Auto Loan Calculations
---

**Purpose:**

Review techniques for creating, reading, viewing, and saving data

## Standard Library

In [ ]:
# system libraries
import os
import sys

# check system information
print('Python Information', sys.version)
print('This is your current directory', os.getcwd())

In [ ]:
# datetime libraries
import datetime

# assgin current date and time
current_date = datetime.date.today()
current_time = datetime.datetime.now()

# check datetime information
print('Today is {}'.format(current_date))
print('Today is', datetime.datetime.strftime(current_date, '%m/%d/%Y'))
print('The time is', datetime.datetime.strftime(current_time, '%H:%M:%S'))

In [ ]:
import csv
import json

In [ ]:
def loan_func(present_value, number_periods, interest_rate):
    """
    This function helps us understand PEMDAS order of operations
    Please = parentheses
    Excuse = exponents
    My = multiplication
    Dear = division
    Aunt = addition
    Sally = subtraction
    """
    interest_rate = interest_rate/100/12
    return interest_rate * present_value / (1-(1+interest_rate)** -number_periods)

loan_func(25515, 72, 3.19)

In [ ]:
auto_loan = loan_func(25515, 72, 3.19)
auto_loan

In [ ]:
# Also works for mortgages

home_loan = loan_func(450000, 180, 2.5)
home_loan

In [ ]:
number_periods_list = [t for t in range(24,73,12)]

for n in number_periods_list:
    print(n)

In [ ]:
current_date = datetime.datetime.now().strftime(format='%Y-%m-%d %H:%M:%S')

print(current_date)

In [ ]:
interest_rate = 4.7
present_value = 25000
data = []

while (present_value < 30000):
    
    for number_periods in number_periods_list:
        
        monthly_payment = loan_func(present_value, number_periods, interest_rate)
        
        monthly_payment_dict = {
            'LOAN': present_value,
            'INTEREST': interest_rate,
            'TERM': number_periods,
            'MONTHLY': round(monthly_payment, 2),
            'DATE': current_date,
        }
        
        data.append(monthly_payment_dict)
        
    present_value += 1000

In [ ]:
type(data)

In [ ]:
type(data[0])

In [ ]:
for key, value in data[0].items():
    print(key, '>>>', value)

In [ ]:
data[0:2]

In [ ]:
fieldnames = data[0].keys()
fieldnames

In [ ]:
# save as CSV (Comma Separated Values)
with open(file='loans.csv', mode='w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data)

In [ ]:
# save as JSON (JavaScript Object Notation)
with open(file='loans.json', mode='w') as fp:
    json.dump(fp=fp, obj=data, indent=2)

## SQL

In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float

In [ ]:
# This creates a database (if it does not exist) or opens the existing one
engine = create_engine('sqlite:///loans.db', echo=False)

In [ ]:
# These steps create the loans table based on the data formats imported from the sqlalchemy library

meta = MetaData()

loans = Table(
    'loans', meta,
    Column('LOAN', Integer),
    Column('INTEREST', Float),
    Column('TERM', Integer),
    Column('MONTHLY', Float),
    Column('DATE', String),
)

meta.create_all(engine)

conn = engine.connect()

In [ ]:
# We can use the LIST of DICTIONARIES we created using the Standard Library
for d in data:
    print(d)

In [ ]:
# SQL Alchemy is able to interpret this format for inserting records to the table

conn.execute(loans.insert(), data)

## Numpy

In [ ]:
import numpy as np

In [ ]:
rate = 4.7
rate

In [ ]:
term_list = np.linspace(start=24, stop=72, num=5, dtype='int8').reshape(-1, 1)
term_list

In [ ]:
loan_amount = np.arange(start=25000, stop=32000, step=1000, dtype='int64')
loan_amount

In [ ]:
monthly_payment = np.pmt(rate=rate/100/12, nper=term_list, pv=-loan_amount).round(2)
monthly_payment

In [ ]:
!pip install numpy_financial

In [ ]:
import numpy_financial as npfin

In [ ]:
monthly_payment = npfin.pmt(rate=rate/100/12, nper=term_list, pv=-loan_amount).round(2)
monthly_payment

## Pandas

In [ ]:
import pandas as pd

In [ ]:
df_sql = pd.read_sql_table(table_name='loans', con=conn)
df_sql.head()

In [ ]:
sql = """
SELECT *
FROM loans
WHERE MONTHLY < 550
AND DATE = (SELECT MAX(DATE) FROM loans)
ORDER BY MONTHLY DESC
"""

df_qry = pd.read_sql_query(sql=sql, con=conn)
df_qry

In [ ]:
conn.close()

In [ ]:
df_csv = pd.read_csv('loans.csv')
df_csv.head()

In [ ]:
df_json = pd.read_json('loans.json')
df_json.head()

In [ ]:
df = pd.DataFrame(monthly_payment)
df

In [ ]:
term_list.ravel()

In [ ]:
df.index = term_list.ravel()
df

In [ ]:
df.columns = loan_amount
df

In [ ]:
df.loc[60, 28000]

In [ ]:
xf = df.reset_index()\
    .melt(id_vars='index', value_name='MONTHLY', var_name='LOAN')\
    .assign(INTEREST=rate, DATE=current_date)\
    .rename(columns={"index": "TERM"})

xf

In [ ]:
xf.to_excel('loans.xlsx', index=None)

## Seaborn

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(xf['MONTHLY']);

In [ ]:
sns.boxplot(x='LOAN', y='MONTHLY', data=xf);